## Order of Presenting Works

For Q1, I have tried five methods to solve it. In order to let the code run from top to down, I will present my last version first, which performed best and put my four other versions of code at the bottom. Altough it performed best, it cannot always find the shortest path.

## Idea and Definitions
My idea came from this paper: Fast Exact Shortest-Path Distance Queries on Large Networks by Pruned Landmark Labeling, which I will refer as `[1]` below. The idea is based on two-hop cover and two-hop labels. Basically, a two-hop cover has a center node and two node sets, one containing nodes that are reachable to center and the shortest distance to it, and one containing nodes and shortest distance that are reachable from center. Two-hop labels are a table of the set of two-hop covers that contains all the nodes reachability and shortest distance. We check two-hop labels table to respond shortest distance query.

![q1.1](./q1.1.PNG)

### Example
In this eample, (a) is our graph, where the weight of edges are all 1. 
(b) and (c) are two two-hop covers, which is consisted by three node layers. The top layer represents nodes that can reach the center node, which is the only node on the middle layer. The bottom layer contains nodes that can be accessed from the center node. The distance between layers are all shortest distance. 
(d) is the two-hop labels, which froms by two-hop covers. `L(ni)` stands for in and out labels of node `ni`. Two-hop labels are consisted by an in edge and out edge sets where edges are represented by another end of the node and the shortest distance between the node and label node. 

For instance, to find a path from `n3` to `n15`, we check out edges from `n3`, which is the second set in `L(n3)`, and in edges to `n15`, which is the first set in `L(15)`. If we find `n15` in out edges of `n3` or `n3` in in edges of `n15`, then we find the shorest path between them, because two-hop cover only stores the shortest path. In this case, we find `<n15: 2>` in `L(n3)`, which means the shortest path from `n3` to `n15` is 2. Otherwise, the overlapping nodes are potential shortest paths passing through them. We need to add distance from source node to overlapping nodes and distance from overlapping nodes to destionation nodes and get the minimum distance. for example, if we want to find the shortest distance between `n0` and `n7`, we find that `n8` and `n15` are both out edge of `n0` and in edge of `n7`. We calculate the shortest path from `n0` to `n7` passing `n8` and `n15` seperately. 2+2 < 3+2, thus the shortest distance from `n0` to `n7` is 4 passing thourgh `n8`.

## Code
This section is simply importing some modules.

In [ ]:
from collections import deque
from math import inf

### Algorithm
Our goal is to construct a two-hop labels. To achieve this, we do graph traversal from each node to form two-hop covers. A naive way is to do Dijkstra algorithm to find the shortest paths from each center nodes, which is one of my methods and I will mention more details below. However, it has time complexity `O(V^3)`, which is obviously inefficient and impractical. 

`[1]` proposed a pruned BFS. The strategy is as follows: Initially, we set distance between all nodes to infinity. While doing BFS for each center node `v`, when visiting a node `u`, we query two-hop labels the distance from `v` to `u`. If the query distance is bigger than distance from current BFS, we add edge `(v, u)` into two-hop labels and append `u` into queue for traversal. Otherwise, we start to visit the next node in the queue.

![q1.2](./q1.2.PNG)

### Two-Hop Cover
Below is the initialization of my two-hop cover class. It takes graph G, center node and two-hop labels as arguments. `labels` will be updated thrgouhout finding two-hop covers. The structure of `labels` will be introduced in `Shortest Distance Section`. Notice that two-hop cover need to do two graph traversal, one for outgoing edges and one for incoming edges.

### Pruned BFS
This section will briefly explain my code. In general, my code follows the Algorithm 1 above. The argument `direction` is either `"in"` or `"out"`. When direction is `"out"`, it does outgoing traversal, which is the same as normal traversal. When direction is `"in"`, it traverses through reverse edges.

In the pruning precess, I add lines to compare original neighbor's distance to current distance and update it, which is different from `[1]`. This is because `[1]` assume all edges have same weights, thus no needs to update for different edge weights.

The function `query_with_label()` is the same as the query function for doing final query except it use labels from argument and it reverse the label direction if direction is `"in"`.

In [ ]:
class TwoHopCover():
    def __init__(self, G, center, labels):
        self.G = G
        self.center = center
        self.prunedBFS(labels, 'out')
        self.prunedBFS(labels, 'in')
            
    def prunedBFS(self, labels, direction):
        # Some initialization
        G = self.G
        src = self.center
        dist = {}
        q = deque()
        q.append(src)
        for v in G.vertex_dict:
            dist[v] = inf
        dist[src] = 0

        # Pruned BFS
        while len(q) != 0:
            u = q.popleft()
            # Pruning
            if self.query_with_label(src, u, labels, direction) > dist[u]:
                labels[src][direction][u] = dist[u]

                # Use reverse edges if direction is "in"
                if direction == 'out':
                    edges = G.adj_list_out[G.vertex_dict[u]]
                else:
                    edges = G.adj_list_in[G.vertex_dict[u]]

                for edge in edges:
                    v = edge[0]
                    v_dist = edge[1]
                    if dist[v] == inf:
                        q.append(v)
                    
                    # Different from the paper: update weight to neighbor if current path have shorter distance
                    if dist[v] > dist[u] + v_dist:
                        dist[v] = dist[u] + v_dist
        return
                
    # Query function used in pruned BFS
    def query_with_label(self, source_vertex, target_vertex, labels, direction):
        reverse_direction = 'in' if direction == 'out' else 'out'
        out_label = {out_v for out_v in labels[source_vertex][direction]}
        in_label = {in_v for in_v in labels[target_vertex][reverse_direction]}

        # If target node is in source node's label or vise versa, immediatly return the shortest distance
        if source_vertex in in_label:
            return labels[target_vertex][reverse_direction][source_vertex]
        if target_vertex in out_label:
            return labels[source_vertex][direction][target_vertex]
        
        # Else we need to find the middle node.
        dist = inf
        for mid_v in out_label & in_label:
            curr_dist = labels[source_vertex][direction][mid_v] + labels[target_vertex][reverse_direction][mid_v]
            if curr_dist < dist:
                dist = curr_dist
        return dist

![q1.3](./q1.3.PNG)

### Shortest Distance
This section I will introduce how I preprocess the graph by two-hop cover class above.
`labels` is a dictionary which contains vertices as keys matching the two-hop cover as the keys as center nodes. The two-hop covers is a dictionary that have two keys, `"in"` and `"out"` which seperately store in and out distance to reachable nodes. for example, there is edges from `A` to `C` with weight `2` and `A` to `D` with weight `6`. labels would be like this: 
`{
    "A": {
        "in": {}, 
        "out": {"C": 2, "D": 6}
    }
    "C": {
        "in": {"A": 2}
        "out": {}
    }
    "D": {
        "in": {"A": 6}
        "out": {}
    }
}`
### Preprocess
The preprocess function basically follows Algorithm 2 above. The difference is the order of nodes to calculate two-hop cover. Though Algorithm 2 choose node randomly, `[1]` mentioned the order of center node is also a crucial factor. The less nodes we add into two-hop labels, the less redundent our table is. 

Thus it is better to start from nodes that its two-hop cover covers the most short edges, and end at nodes that its cover covers the most redundent edges. In my code, I process nodes that have higher degree first, becaus they are more likely to contain short edges.

In [ ]:
class ShortestDistance(object):
    def __init__(self, G):
        self.G = G
        vertices = list(G.vertex_dict.keys())
        self.labels = {v: {'in': {}, 'out': {}}  for v in vertices}
        self.preprocess(G, self.labels)
            
    def preprocess(self, G, labels):
        labels = self.labels
        degrees = self.get_degrees(G)
        
        # While there is node haven't been processed
        while len(degrees) != 0:
            curr_degree = -1
            curr_v = ''

            # Find node with highest degree
            for v in degrees:
                next_degree = degrees[v]
                if next_degree >= curr_degree:
                    curr_degree = next_degree
                    curr_v = v
            if curr_v == '':
                break
            degrees.pop(curr_v)
            TwoHopCover(G, curr_v, labels)
        return 

    # Similar to query_with_label above
    def query(self, source_vertex, target_vertex):
        labels = self.labels
        out_label = {out_v for out_v in labels[source_vertex]['out']}
        in_label = {in_v for in_v in labels[target_vertex]['in']}

        if source_vertex in in_label:
            return labels[target_vertex]['in'][source_vertex]
        if target_vertex in out_label:
            return labels[source_vertex]['out'][target_vertex]

        dist = inf
        for mid_v in out_label & in_label:
            curr_dist = labels[source_vertex]['out'][mid_v] + labels[target_vertex]['in'][mid_v]
            if curr_dist < dist:
                dist = curr_dist
        return dist

    # Given graph G, return a dictionary that vertex is key and degree of vertex is value
    def get_degrees(self, G):
        vertices = list(G.vertex_dict.keys())
        d = {v: (len(G.adj_list_in[G.vertex_dict[v]]) + len(G.adj_list_out[G.vertex_dict[v]])) for v in vertices}
        return d

### Time and Space Complexity

The time and space complexity for pruned BFS is `O(m+n)`. In preprocessing, we do `n` times pruned BFS, so the time complexity for preprocessing is `O(n(m+n))`. However, in practice, we rarely reach to the worst case, especially for pruned BFS, because as we store more labels, the less later nodes can traverse.

## 2. Graph Data Structure
Below is the data stucture of the input graph `G` in the `ShortestDistance` class.

In [ ]:
############################################################################
# Do not edit this code cell.
############################################################################

class DirectedWeightedGraph(object):
  def __init__(self, edge_list):
      self.vertex_dict = {}
      self.adj_list_in = []
      self.adj_list_out = []
      self.vertex_num = 0
      for [src, dst, weight] in edge_list:
          self.add_edge(src, dst, weight)

  def add_vertex(self, name):
      id = self.vertex_num
      self.vertex_dict[name] = id
      self.vertex_num += 1
      self.adj_list_in.append(list())
      self.adj_list_out.append(list())

  def add_edge(self, vertex1, vertex2, weight):
      if vertex1 not in self.vertex_dict.keys():
          self.add_vertex(vertex1)
      if vertex2 not in self.vertex_dict.keys():
          self.add_vertex(vertex2)
      self.adj_list_out[self.vertex_dict[vertex1]].append([vertex2, weight])
      self.adj_list_in[self.vertex_dict[vertex2]].append([vertex1, weight])

## 3. How to test your code

### 3.1 Download the sample dataset.

Running the following command will create a folder COMP9312_datasets containing files about three datasets. **Cora** (2k vertices) is a real citation graph with an synthetic random weight for each edge. **map_BJ_part** (4k vertices) is a real road network for a small area of Beijing, and **map_NY_part** (7k vertices) is a real road network for a small area of New York. Each dataset has three files. For each dataset, ***.graph** includes all graph edges. ***.query** includes a set of shortest distance queries for testing. ***.answer** includes the answer for each query computed by us in the ***.query** file for your reference.

If the dataset has already exists, there would be an error like "*destination path 'COMP9312_datasets' already exists*".

**Note**: We will use different query datasets to test the algorithm.

In [ ]:
!git clone https://github.com/guaiyoui/COMP9312_datasets.git

### 3.2 The main function

Our test procedure first loads the graph dataset and the query dataset. Then it calls the `ShortestDistance` class to preprocess the graph. After that, it will run each query and test their efficiency and correctness. 

In [ ]:
import time
import numpy as np

if __name__ == "__main__":

    print('\n##### Loading the dataset...')
    # edge_list = np.loadtxt('./COMP9312_datasets/cora.graph', dtype=int)
    # query_list = np.loadtxt('./COMP9312_datasets/cora.query', dtype=int)
    edge_list = np.loadtxt('./COMP9312_datasets/map_BJ_part.graph', dtype=int)
    query_list = np.loadtxt('./COMP9312_datasets/map_BJ_part.query', dtype=int)
    # edge_list = np.loadtxt('./COMP9312_datasets/map_NY_part.graph', dtype=int)
    # query_list = np.loadtxt('./COMP9312_datasets/map_NY_part.query', dtype=int)
    G = DirectedWeightedGraph(edge_list)

    print('\n##### Start to preprocessing...')
    start_preprocessing = time.time()
    SD = ShortestDistance(G)
    end_preprocessing = time.time()
    print("preprocessing time: {}".format(end_preprocessing-start_preprocessing))

    print('\n##### Test on the query ...')
    start_query = time.time()
    for i in range(query_list.shape[0]):
      distance = SD.query(query_list[i][0], query_list[i][1])
      print("the distance between {} and {} is: {}".format(query_list[i][0], query_list[i][1], distance))
    end_query = time.time()
    print("average  query time: {}".format((end_query-start_query)/query_list.shape[0]))

## Appendix

In this section, I will introduce my other thoughts in regard to Q1. All my methods did not change graph structure, so I will not repeatedly provide graph class in my code. I did not change `DirectedWeightedGraph`. To test them, simply uncomment and run them. Some of my code is incomplete and some are inefficient. In order not to cause any errors from them, I will make them comment.

## Method 1

The idea came from this paper: On-line Exact Shortest Distance Query Processing, which will be referred to as `[2]` below. The method is two-hop cover based, which is similar to the above method. Apart from that, `[2]` exploit strongly connected components, or simply SCCs, and DAG components and partition graphs to speed up the preprocessing. Besides, it proposed a method to reduce the redundancy of two-hop cover.

### Algorithm

The Algorithm shown below, called DAPar, contains two main parts. First (lines 1 to 7), collapse SCCs into DAGs by removing some nodes. Next (lines 8 to 15), the graph will be partitioned into small subgraphs by removing some nodes until we obtain two hop covers for the whole graph. Two-hop covers are computed when removing nodes.

![q1.4](./q1.4.PNG)

The figure below shows the whole preprocessing process. Figure (a), (b) and (c) match the first step. Figure (d) and (e) match the second step.

![q1.5](./q1.5.PNG)

### Step One: Collapse SCC to DAG

In this section, I will discuss how SCCs collapse to DAGs.

The first thing that should be done is to break the cycles. The optimal solution is to remove the minimum number of nodes to break all the cycles. To avoid high consumption of computation power, `[2]` did not break all cycles at once. Instead, it used an early stop strategy. That is, first, get a fixed amount of cycles. Second, find the minimum number of nodes that break current cycles. Then, repeat the first and second steps.

After collapsing SCCs, we need to get the two-hop covers for removed nodes. Different from `[1]`, it forms two-hop covers according to Algorithm 2 below. The detailed method to compute two-hop covers in `[2]` is Dijkstra Algorithm.

![q1.6](./q1.6.PNG)

### Step Two: Top-down Partitioning

To partition the graph, we find a set of so-called node-separator `Vw` and remove them from the graph. `[2]` introduced two strategies, the fixed strategy for dense graphs and the flexible strategy for sparse graphs. Here, I will only talk about the fixed strategy.

For the fixed strategy, previous studies have proved that with the same space cost, a two-hop cover covers more paths when the cardinality of ancestor nodes and descendent nodes are more similar. That is, balanced 2-hop clusters are preferred. Therefore, the strategy found a node-separator from the middle of the graph.

### Time and Space Complexity

In terms of time complexity, step one takes `O(m(m+n))` times to collapse SCCs. However, the early stop strategy can speed up a lot. In step two, it takes `O(m)` to partition all nodes. It takes `O(n^2)` time to compute two-hop covers by running Dijkstra. In all, time complexity is `O(m^2+n^2+mn)`. The space complexity for two-hop cover is `O(n^2)`, if every cover stores all other nodes.

### Code (incomplete)

Below is my code for this method. Due to time limitations and the complexity of the algorithm, I did not finish it. The function `get_strongly_connected_components()` is able to return all SCCs in the graph. However, I stopped at finding cycles in SCCs.

In [ ]:
'''
from collections import namedtuple
import copy

path = namedtuple('path', ['src', 'dest', 'distance'])
early_stop = 30

class ShortestDistance(object):
    def __init__(self, G):
        self.G = G
        self.preprocess(G)

    def preprocess(self):
        G = collapse_G(G)
        split_G(G)

    def query(self, source_vertex, target_vertex):
        shortest_distance = 0
        return shortest_distance

def collapse_G(G):
    SCCs = get_strongly_connected_components()
    for SCC in SCCs:
        cycles = getCycles(G, SCC)
        v_hat = getDAG(cycles)
        for v in v_hat:
            add_two_hop_cover(v)
    G = remove_v(v_hat)
    return G

def split_G(G, v_sep)-> None:
    v_sep = find_separator()
    G_top, G_bot = separate_G(G, v_sep)
    for v in v_sep:
        add_two_hop_cover(v)

    if is_small(G_top):
        for v in G_top:
            add_two_hop_cover(v)
    else:
        split_G(G_top)

    if is_small(G_bot):
        for v in G_bot:
            add_two_hop_cover(v)
    else:
        split_G(G_bot)

def get_strongly_connected_components(G) ->list:
    SCCs = []
    visited = set()
    stack = []
    vertices = list(G.vertex_dict.keys())
    for v in vertices: 
        if v not in visited:
            DFS(G, [], v, visited, stack)
    reverse = reverse_G(G)
    visited = set()
    while stack:
        v = stack.pop()
        if v not in visited:
            component = DFS(reverse, [], v, visited, [])
            SCCs.append(component)
    return SCCs

def getCycles(G, cycles):
    
def getDAG(cycles: list):
    current_cycle = cycles
    v_hat = set()
    if len(cycles) > early_stop:
        current_cycle = cycles[:early_stop]
        next_cycle = cycles[early_stop:]
        v_hat += getDAG(next_cycle)
    
    for vertices in cycles:
        v_total += set(vertices)
    v_appear_count = {}
    for v in v_total:
        for c in cycles

def add_two_hop_cover(v):
    pass
def remove_v(v_hat): 
    pass
def find_separator():
    pass
def separate_G(v_sep):
    pass
def is_small(G):
    pass

def DFS(G, traversal, vertex, visited, stack):
    visited.add(vertex)
    traversal.append(vertex)
    neighbors = G.adj_list_out[G.vertex_dict[vertex]]
    for out_edge in neighbors:
        v = out_edge[0]
        if v not in visited:
            DFS(G, traversal, v, visited, stack)
    stack.append(vertex)
    return traversal

def reverse_G(G):
    r_G = copy.deepcopy(G)
    temp = copy.deepcopy(r_G.adj_list_in)
    r_G.adj_list_in = r_G.adj_list_out
    r_G.adj_list_out = temp
    return r_G
'''

## Method 2

The idea came from the lecture Path and Reachability. Inspired by the two-hop cover method in the lecture, I tried to implement it to find the shortest distance. My two-hop cover only considered the neighbor nodes, it did a recursive DFS when querying. 

### Time and Space Complexity

The preprocessing space complexity for two-hop cover is `O(n^2)`. The time complexity for that is `O(n^2)`. The query time complexity is `O(m)`. In practice, the preprocessing speed is fast. But the query time is not acceptable.

### Code (inefficient)

The code for this method is completed, but the query time is quite long for big graph.

In [ ]:
'''
from math import inf
import copy

class ShortestDistance(object):
    def __init__(self, G):
        self.G = G
        self.labels_in, self.labels_out = self.preprocess(G)


    def preprocess(self, G: DirectedWeightedGraph):
        labels_in = []
        labels_out = []
        for v in range(G.vertex_num):
            labels_in.append([]) 
            labels_out.append([]) 

        degree_pairs = self.get_graph_degree(G)
        highest_degree = self.get_highest_degree(degree_pairs)
        while highest_degree >= 0:
            for d_pair in degree_pairs:
                if highest_degree == d_pair[1]:
                    curr_v = d_pair[0]
                    d_pair[1] = -1
                    break
            for in_edge in G.adj_list_in[G.vertex_dict[curr_v]]:
                in_v = in_edge[0]
                in_dist = in_edge[1]

                (labels_out[G.vertex_dict[in_v]]).append([curr_v, in_dist])
                for out_label in labels_out[G.vertex_dict[in_v]]:
                    mid_v = out_label[0]
                    for in_label in labels_in[G.vertex_dict[curr_v]]:
                        potential_mid_v = in_label[0]
                        if mid_v == potential_mid_v:
                            if out_label[1] + in_label[1] > in_dist:
                                labels_out[G.vertex_dict[in_v]].remove(out_label)
                                labels_in[G.vertex_dict[curr_v]].remove(in_label)
                            elif out_label[1] + in_label[1] < in_dist:
                                labels_out[G.vertex_dict[in_v]].remove([curr_v, in_dist])



            for out_edge in G.adj_list_out[G.vertex_dict[curr_v]]:
                out_v = out_edge[0]
                out_dist = out_edge[1]
                
                labels_in[G.vertex_dict[out_v]].append([curr_v, out_dist])
                for in_label in labels_in[G.vertex_dict[out_v]]:
                    mid_v = in_label[0]
                    for out_label in labels_out[G.vertex_dict[curr_v]]:
                        potential_mid_v = out_label[0]
                        if mid_v == potential_mid_v:
                            if in_label[1] + out_label[1] > out_dist:
                                labels_in[G.vertex_dict[out_v]].remove(in_label)
                                labels_out[G.vertex_dict[curr_v]].remove(out_label)
                            elif in_label[1] + out_label[1] < out_dist:
                                labels_in[G.vertex_dict[out_v]].remove([curr_v, out_dist])
            
            highest_degree = self.get_highest_degree(degree_pairs)

        return (labels_in, labels_out)

    def query(self, source_vertex, target_vertex):
        vertices = list(self.G.vertex_dict.keys())
        return self.recQuery(source_vertex, target_vertex, 0, vertices)
    
    def recQuery(self, src, dest, dist, unvisited):
        if len(unvisited) == 0:
            return inf

        G = self.G
        labels_in = self.labels_in
        labels_out = self.labels_out

        for out_edge in labels_out[G.vertex_dict[src]]:
            out_v = out_edge[0]
            out_dist = out_edge[1]
            if out_v == dest:
                dist += out_dist
                return dist
                
        for in_edge in labels_in[G.vertex_dict[dest]]:
            in_v = in_edge[0]
            in_dist = in_edge[1]
            if in_v == src:
                dist += in_dist
                return dist
        
        curr_dist = inf
        for out_edge in labels_out[G.vertex_dict[src]]:
            out_v = out_edge[0]
            out_dist = out_edge[1]
            try:
                unvisited.remove(out_v)
                curr_dist = min(self.recQuery(out_v, dest, dist, copy.copy(unvisited)) + out_dist, curr_dist)
            except:
                pass

        for in_edge in labels_in[G.vertex_dict[dest]]:
            in_v = in_edge[0]
            in_dist = in_edge[1]
            try:
                unvisited.remove(in_v)
                curr_dist =  min(self.recQuery(src, in_v, dist, copy.copy(unvisited)) + in_dist, curr_dist)
            except:
                pass
        
        return curr_dist
        

    def get_graph_degree(self, G: DirectedWeightedGraph):
        degrees = []
        vertices = list(G.vertex_dict.keys())
        for v in vertices:
            d = len(G.adj_list_in[G.vertex_dict[v]]) + len(G.adj_list_out[G.vertex_dict[v]])
            d_pair = [v, d]
            degrees.append(d_pair)
        return degrees

    def get_highest_degree(self, degree_pairs: list):
        degrees = []
        for d_pair in degree_pairs:
            degrees.append(d_pair[1])
        return max(degrees)
        '''

## Method 3

The idea came from this paper: Reachability and distance query via 2-hop labels, which will be referred to as `[3]`. It is also based on a two-hop cover. The problem it resolved is to find an optimal two-hop cover set. The strategy is before all nodes are covered, evaluate every two-hop cover by a ratio shown below and pick the best one. Add the cover to two-hop labels. In this way, the two-hop labels are close to optimal.

![q1.7](./q1.7.PNG)

Here, w is the center node. C_in and C_out are the set of nodes that have an edge goes into and out w. S(C_in, w, C_out) represent the two-hop cover. T' means the residual edges that haven't been covered. |C_in| and |C_out| are numbers of nodes in sets.

### Time and Space Complexity

Although it can obtain a nearly optimal two-hop label table, the preprocessing space complexity is `O(n^2)` for two-hop covers, and `O(m)` for T'. In terms of time complexity, when preprocessing, every round we need to spend `O(m^2)` time to get the intersection of cover and `T`, then calculate the ratio for each two-hop cover one by one, which is `O(m^2n)` in all. The space complexity for labels is `O(m)`. In practice, the preprocessing time is quite large due to Dijkstra for every node and calculation ratio.

### Code (Inefficient)

The code is able to work, but the preprocessing time is quite long and impractical.

In [ ]:
'''
from itertools import permutations
from math import inf
import copy

class TwoHopCover():
    def __init__(self, G, center) -> None:
        self.center = center
        self.desc = self.dijkstra(G, center, 'out')
        self.ancs = self.dijkstra(G, center, 'in')
        
    def cover(self):
        return [self.ancs, self.desc]

    def dijkstra(self, G: DirectedWeightedGraph, src, direction='out'):
        dist = {}
        prev = {}
        q = list(G.vertex_dict.keys())
        for v in G.vertex_dict:
            dist[v] = inf
            prev[v] = ''
        dist[src] = 0
        while len(q) != 0:
            u = self.get_u(q, dist)
            if dist[u] == inf:
                break
            q.remove(u)
            if direction == 'out':
                edges = G.adj_list_out[G.vertex_dict[u]]
            else:
                edges = G.adj_list_in[G.vertex_dict[u]]

            for edge in edges:
                v = edge[0]
                v_dist = edge[1]
                if dist[v] > dist[u] + v_dist:
                    dist[v] = dist[u] + v_dist
                    prev[v] = u
        for v in list(G.vertex_dict.keys()):
            if dist[v] == inf:
                dist.pop(v)
        dist.pop(src)
        return dist
                
    def get_u(self, q: list, dist: dict) ->str:
        nearest_v = q[0]
        nearest_dist = dist[q[0]]
        for v in q:
            if dist[v] < nearest_dist:
                nearest_v = v
                nearest_dist = dist[v]
        return nearest_v
    
    def cover_weight(self):
        return len(self.ancs) + len(self.desc)
    
    def intersect(self, vertices):
        non_covered_path = list(permutations(vertices, 2))
        num_total = len(non_covered_path)
        self.do_intersect(non_covered_path)
        num_left = len(non_covered_path)
        return num_total - num_left
        
    def do_intersect(self, non_covered_path):
        for in_v in self.ancs:
            ancs_path = (in_v, self.center)
            if ancs_path in non_covered_path:
                non_covered_path.remove(ancs_path)
        for out_v in self.desc:
            desc_path = (self.center, out_v)
            if desc_path in non_covered_path:
                non_covered_path.remove(desc_path)
            
    

class ShortestDistance(object):
    def __init__(self, G):
        self.G = G
        vertices = list(G.vertex_dict.keys())
        self.labels = {v: {'in': {}, 'out': {}}  for v in vertices}
        self.preprocess(G)


    def preprocess(self, G: DirectedWeightedGraph):
        vertices = list(G.vertex_dict.keys())
        stored = set()
        non_covered_path = list(permutations(vertices, 2))
        cover_list = {v: TwoHopCover(G, v) for v in vertices}
        curr_ratio = 0
        curr_v = ''
        while len(non_covered_path) != 0:
            for v in cover_list:
                next_ratio = cover_list[v].intersect(vertices) / cover_list[v].cover_weight()
                if next_ratio >= curr_ratio and v not in stored:
                    curr_ratio = next_ratio
                    curr_v = v
            
            self.store_labels(cover_list[curr_v])
            stored.add(curr_v)
            prev_non_covered_path = copy.copy(non_covered_path)
            cover_list[curr_v].do_intersect(non_covered_path)
            if prev_non_covered_path == non_covered_path:
                break
        return 

    def query(self, source_vertex, target_vertex):
        labels = self.labels
        out_label = {out_v for out_v in labels[source_vertex]['out']}
        in_label = {in_v for in_v in labels[target_vertex]['in']}

        if source_vertex in in_label:
            return labels[target_vertex]['in'][source_vertex]
        if target_vertex in out_label:
            return labels[source_vertex]['out'][target_vertex]

        dist = inf
        for mid_v in out_label & in_label:
            curr_dist = labels[source_vertex]['out'][mid_v] + labels[target_vertex]['in'][mid_v]
            if curr_dist < dist:
                dist = curr_dist
        return dist
    
    def store_labels(self, cover: TwoHopCover):
        v = cover.center
        labels = self.labels
        for in_v in cover.ancs:
            labels[in_v]['out'][v] = cover.ancs[in_v]
        for out_v in cover.desc:
            labels[out_v]['in'][v] = cover.desc[out_v]
'''

## Mathod 4

This method simply implemented two-hop cover with no optimization. For the order of nodes to preprocess, I simply choose degree. Starting from the node with the highest degree, it compute two-hop cover by Dijkstra algorithm and store labels into the table. then it find the node with second high degree and repeat the process, until no new edge is stored in a round or all nodes have been processed.

### Time and Space Complexity

As above mentioned, it takes `O(n^3)` time `O(n^2)` space to compute `n` two-hop covers. The preprocessing process does at most `n` rounds to remove at most `n` edges in each round. Thus the time complexity is `O(n^3)` in all. In practice, the process of computing two-hop cover is quite time-consuming, though it gives the correct shortest distance.

### Code (Inefficient)

The code is completed and faster than mathod 2 and 3. However, it is still unacceptable.

In [ ]:
'''
from math import inf
import copy

class TwoHopCover():
    def __init__(self, G, center) -> None:
        self.center = center
        self.max_edge = round(get_num_edges(G))
        self.desc = self.dijkstra(G, center, 'out')
        self.ancs = self.dijkstra(G, center, 'in')
        
    def cover(self):
        return [self.ancs, self.desc]

    def dijkstra(self, G: DirectedWeightedGraph, src, direction='out'):
        dist = {}
        edge_count = 0
        q = list(G.vertex_dict.keys())
        for v in G.vertex_dict:
            dist[v] = inf
        dist[src] = 0
        while len(q) != 0 and edge_count < self.max_edge:
            edge_count += 1
            u = self.get_u(q, dist)
            if dist[u] == inf:
                break
            q.remove(u)
            if direction == 'out':
                edges = G.adj_list_out[G.vertex_dict[u]]
            else:
                edges = G.adj_list_in[G.vertex_dict[u]]

            for edge in edges:
                v = edge[0]
                v_dist = edge[1]
                if dist[v] > dist[u] + v_dist:
                    dist[v] = dist[u] + v_dist
        for v in list(G.vertex_dict.keys()):
            if dist[v] == inf:
                dist.pop(v)
        dist.pop(src)
        return dist
                
    def get_u(self, q: list, dist: dict) ->str:
        nearest_v = q[0]
        nearest_dist = dist[q[0]]
        for v in q:
            if dist[v] < nearest_dist:
                nearest_v = v
                nearest_dist = dist[v]
        return nearest_v
    
        
    def do_intersect(self, uncovered_path):
        count = 0
        center = self.center
        for in_v in self.ancs:
            try: 
                uncovered_path[in_v].remove(center)
                count += 1
            except:
                pass
                
        for out_v in self.desc:
            try:
                uncovered_path[center].remove(out_v)
                count += 1
            except:
                pass
        return count
            
    

class ShortestDistance(object):
    def __init__(self, G):
        self.G = G
        vertices = list(G.vertex_dict.keys())
        self.labels = {v: {'in': {}, 'out': {}}  for v in vertices}
        self.min_removed_edge = 0
        self.preprocess(G)



    def preprocess(self, G: DirectedWeightedGraph):
        vertices = list(G.vertex_dict.keys())
        degrees = self.get_degrees(G)
        uncovered_path = {v: copy.copy(vertices) for v in vertices}
        for v in vertices: 
            uncovered_path[v].remove(v) 
        rounds = 0
        num_total_removed_edges = 0
        print('removed edge number: {}'.format(num_total_removed_edges))
        while self.get_num_path(uncovered_path) >= 0:
            rounds += 1
            print('rounds: {}'.format(rounds))
            curr_degree = 0
            curr_v = ''
            for v in degrees:
                next_degree = degrees[v]
                if next_degree >= curr_degree:
                    curr_degree = next_degree
                    curr_v = v
                if curr_v == '':
                    break
            print('found curr_v: {}'.format(curr_v))
            degrees.pop(curr_v)
            curr_cover = TwoHopCover(G, curr_v)
            self.store_labels(curr_cover)
            num_remove_edges = curr_cover.do_intersect(uncovered_path)
            num_total_removed_edges += num_remove_edges
            if num_remove_edges <= self.min_removed_edge:
                print('stop. current edges: {}'.format(num_total_removed_edges))
                break
            print('reomve {} edges. current edges: {}'.format(num_remove_edges, num_total_removed_edges))
        return 

    def query(self, source_vertex, target_vertex):
        labels = self.labels
        out_label = {out_v for out_v in labels[source_vertex]['out']}
        in_label = {in_v for in_v in labels[target_vertex]['in']}

        if source_vertex in in_label:
            return labels[target_vertex]['in'][source_vertex]
        if target_vertex in out_label:
            return labels[source_vertex]['out'][target_vertex]
        dist = inf
        for mid_v in out_label & in_label:
            curr_dist = labels[source_vertex]['out'][mid_v] + labels[target_vertex]['in'][mid_v]
            if curr_dist < dist:
                dist = curr_dist
        return dist
    
    def get_num_path(self, uncovered_path):
        count = 0
        for v in uncovered_path:
            count += len(uncovered_path[v])
        return count

    def store_labels(self, cover: TwoHopCover):
        v = cover.center
        labels = self.labels
        labels[v]['in'][v] = 0
        labels[v]['out'][v] = 0
        in_label = {in_v for in_v in labels[v]['in']}
        for in_v in cover.ancs:
            out_label = {out_v for out_v in labels[in_v]['out']}
            if not (in_label & out_label):
                labels[in_v]['out'][v] = cover.ancs[in_v]
            else:
                for mid_v in (in_label & out_label):
                    if labels[in_v]['out'][mid_v] + labels[v]['in'][mid_v] > cover.ancs[in_v]:
                        labels[in_v]['out'][v] = cover.ancs[in_v]

        out_label = {out_v for out_v in labels[v]['out']}
        for out_v in cover.desc:
            in_label = {in_v for in_v in labels[out_v]['in']}
            if not (in_label & out_label):
                labels[out_v]['in'][v] = cover.desc[out_v]
            else:
                for mid_v in (in_label & out_label):
                    if labels[v]['out'][mid_v] + labels[out_v]['in'][mid_v] > cover.desc[out_v]:
                        labels[out_v]['in'][v] = cover.desc[out_v]
    
    def get_degrees(self, G: DirectedWeightedGraph):
        vertices = list(G.vertex_dict.keys())
        d = {v: (len(G.adj_list_in[G.vertex_dict[v]]) + len(G.adj_list_out[G.vertex_dict[v]])) for v in vertices}
        return d

    
def get_num_edges(G):
    count = 0
    for in_edges in G.adj_list_in:
        count += len(in_edges)
    return count
'''

## Reference
`[1].` Akiba, T., Iwata, Y., & Yoshida, Y. (2013, June). Fast exact shortest-path distance queries on large networks by pruned landmark labeling. In Proceedings of the 2013 ACM SIGMOD International Conference on Management of Data (pp. 349-360).

`[2].` Cheng, J., & Yu, J. X. (2009, March). On-line exact shortest distance query processing. In Proceedings of the 12th International Conference on Extending Database Technology: Advances in Database Technology (pp. 481-492).

`[3].` Cohen, E., Halperin, E., Kaplan, H., & Zwick, U. (2003). Reachability and distance queries via 2-hop labels. SIAM Journal on Computing, 32(5), 1338-1355.